本文件将用于从多个vtuber数据网站爬取相关数据并存储。

This file will be used to crawl and store relevant data from multiple vtuber data sites.

# vtbs.fun

爬取该网页上所有虚拟主播近一年的相关信息及数据

Crawl the information and data of all the virtual anchors on this page for the past one year

In [27]:
import requests    #导入requests包
import json
import pandas as pd
from bs4 import BeautifulSoup

headers={
    'Cookie': 'sidebarStatus=0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.76',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Connection': 'keep-alive'
}


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jan 14 10:30:03 2023

@author: JYZ
"""



def main():
    for i in range(20000,25000):
        url1='http://www.vtbs.fun:8050/vtb/'+str(i)
        url2='http://www.vtbs.fun:8050/live/liveAnalyse?vid='+str(i)+'&startTime=2022-10-14+21:30:12&endTime=2023-01-12+21:30:12'
        fun1(url1,url2,i)

def fun1(url1,url2,j):
    r_per=requests.get(url1,headers=headers)
    r_live=requests.get(url2,headers=headers)
    
    #个人信息-personal_data
    datas=json.loads(r_per.text)
    if len(datas)==3:
        print('第'+str(j)+'条数据为空白')
    else:
        data_per = datas['data']
        personal_data = pd.DataFrame(columns=data_per.keys())
        personal_data = personal_data.append(data_per,ignore_index=True) 
        #personal_data
        
        #直播总数据-overview_data
        live=json.loads(r_live.text)['data']
        overview=live['overview'] #总数据
        livecount=int(overview['liveCount'])
        overview['participant']*=livecount
        overview['danmaku']*=livecount
        overview['income']*=livecount
        overview['goldUser']*=livecount
        overview_data = pd.DataFrame(columns=overview.keys())
        overview_data = overview_data.append(overview,ignore_index=True) 
        #overview_data
        
        #弹幕前20内容-
        danmu=live['danmakuWordFreqList']
        danmu_times=[]
        danmu_name=[]
        for i in range(0,len(danmu)):
            danmu_name.append(danmu[i]['name'])
            danmu_times.append(danmu[i]['value'])
        danmu=str(dict(zip(danmu_name,danmu_times)))
        danmu_df = pd.DataFrame([danmu], columns=['danmutop20'])
        #danmu_df = pd.DataFrame(danmu)
        #danmu.columns=(danmu_name)
        #danmu=danmu.to_dict('records')
        
        #个人信息+直播信息+弹幕前20及数量
        data=pd.concat([personal_data,overview_data,danmu_df],axis=1)
        global df
        df=pd.concat([df,data],ignore_index=True)
        print('爬取了'+str(j)+"条数据")

def write(df):
    print('开始存储')
    df.to_csv('Crawled_Dataset\虚拟主播1年数据.csv',mode='a',encoding='gb18030')
    print('存储完毕')
    
if __name__ == '__main__':
    df=pd.DataFrame()
    main()
    write(df)

对爬取的数据进行简单处理

basic pre-procession of raw data

In [51]:
df=pd.read_csv("Crawled_Dataset\虚拟主播1年数据.csv",encoding="UTF-8")

In [27]:
#性别列-保密2男1女0
df.loc[df['sex'] == '保密', 'sex'] = '2'
df.loc[df['sex'] == '女', 'sex'] = '0'
df.loc[df['sex'] == '男', 'sex'] = '1'

#官方认证-有1无0
df["official"].fillna("0", inplace = True)
df.loc[df['official'] != '0', 'official'] = '1'

In [29]:
print('开始存储')
df.to_csv('Crawled_Dataset\虚拟主播1年数据.csv',mode='w',encoding='utf-8')
print('存储完毕')

开始存储
存储完毕


# ZERORODU

以天为单位，爬取主播直播间营收数据

Crawling anchor live streaming revenue data by day

In [2]:
import requests    #导入requests包
import json
import pandas as pd
from bs4 import BeautifulSoup

headers={
    'Cookie': 'SESSION=MTY3Mzc5MDYwOHxOd3dBTkZaT1dGaFRTVTB6VUVjMlUwZFRVVXBNVlRjM05FdEZRbFJFVEZJM1Z6TlBSRU5ZTkVKSFNGUk9ORkJJV1VSYU16WlhWVUU9fOLSs37JKPJT4Ld0DkXD4qaGJe_w4ZnWJQqJKsUw5wK7',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.76',
    #'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Connection': 'keep-alive'
}

C:\Users\JYZ\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [1]:
#从爬取的现有数据中，提取主播mid号作为特征，进行下一步爬取
df=pd.read_csv("Crawled_Dataset\虚拟主播1年数据.csv",encoding="UTF-8")
mid=df['mid']
#mid

NameError: name 'pd' is not defined

In [69]:
df=pd.DataFrame()

#单个主播内容
for i in range(0,len(mid)):
    url1='https://api.zeroroku.com/bilibili/author/live?mid='+str(mid[i])
    r_live=requests.get(url1,headers=headers)
    datas=json.loads(r_live.text)
    
    if datas!={"message":"live room not found"}:#部分主播没有收录
        mid_num=[mid[i]]*len(datas)
        df_live=DataFrame(datas)
        df_live['currency']=df_live['currency']/1000 #统一单位
        df_live.insert(0,'mid',mid_num)
        df=df.append(df_live,ignore_index=True)
        print("爬取了第",i,"个主播共",len(datas),"条数据")

#写入excel
print('开始存储')
df.to_csv('Crawled_Dataset\直播间营收数据.csv',mode='w',encoding='gb18030')
print('存储完毕')        

爬取了第 0 个主播共 1 条数据
爬取了第 1 个主播共 4 条数据
爬取了第 3 个主播共 1 条数据
爬取了第 5 个主播共 1 条数据
爬取了第 7 个主播共 1 条数据
爬取了第 8 个主播共 118 条数据
爬取了第 9 个主播共 1 条数据
爬取了第 10 个主播共 2 条数据
爬取了第 11 个主播共 1 条数据
爬取了第 13 个主播共 8 条数据
爬取了第 14 个主播共 1 条数据
爬取了第 15 个主播共 1 条数据
爬取了第 16 个主播共 1 条数据
爬取了第 17 个主播共 1 条数据
爬取了第 18 个主播共 1 条数据
爬取了第 21 个主播共 1 条数据
爬取了第 22 个主播共 20 条数据
爬取了第 23 个主播共 1 条数据
爬取了第 25 个主播共 1 条数据
爬取了第 26 个主播共 1 条数据
爬取了第 27 个主播共 1 条数据
爬取了第 28 个主播共 1 条数据
爬取了第 29 个主播共 1 条数据
爬取了第 31 个主播共 7 条数据
爬取了第 32 个主播共 2 条数据
爬取了第 34 个主播共 3 条数据
爬取了第 35 个主播共 13 条数据
爬取了第 36 个主播共 6 条数据
爬取了第 37 个主播共 3 条数据
爬取了第 38 个主播共 1 条数据
爬取了第 39 个主播共 1 条数据
爬取了第 40 个主播共 1 条数据
爬取了第 43 个主播共 2 条数据
爬取了第 44 个主播共 1 条数据
爬取了第 47 个主播共 8 条数据
爬取了第 48 个主播共 1 条数据
爬取了第 49 个主播共 2 条数据
爬取了第 51 个主播共 1 条数据
爬取了第 52 个主播共 1 条数据
爬取了第 53 个主播共 1 条数据
爬取了第 54 个主播共 1 条数据
爬取了第 55 个主播共 1 条数据
爬取了第 56 个主播共 1 条数据
爬取了第 57 个主播共 25 条数据
爬取了第 59 个主播共 4 条数据
爬取了第 60 个主播共 1 条数据
爬取了第 61 个主播共 3 条数据
爬取了第 63 个主播共 1 条数据
爬取了第 68 个主播共 1 条数据
爬取了第 69 个主播共 1 条数据
爬取了第 70 个主播共 13 条数据
爬取了第 71 个主播共 2 条数据
爬取了第 72 个主播共 

爬取了第 573 个主播共 7 条数据
爬取了第 574 个主播共 4 条数据
爬取了第 575 个主播共 19 条数据
爬取了第 576 个主播共 1 条数据
爬取了第 578 个主播共 1 条数据
爬取了第 579 个主播共 1 条数据
爬取了第 580 个主播共 97 条数据
爬取了第 581 个主播共 9 条数据
爬取了第 582 个主播共 1 条数据
爬取了第 583 个主播共 1 条数据
爬取了第 584 个主播共 3 条数据
爬取了第 585 个主播共 1 条数据
爬取了第 587 个主播共 105 条数据
爬取了第 588 个主播共 4 条数据
爬取了第 589 个主播共 9 条数据
爬取了第 590 个主播共 1 条数据
爬取了第 591 个主播共 1 条数据
爬取了第 592 个主播共 1 条数据
爬取了第 593 个主播共 108 条数据
爬取了第 594 个主播共 1 条数据
爬取了第 595 个主播共 2 条数据
爬取了第 596 个主播共 1 条数据
爬取了第 597 个主播共 1 条数据
爬取了第 598 个主播共 5 条数据
爬取了第 599 个主播共 36 条数据
爬取了第 600 个主播共 1 条数据
爬取了第 601 个主播共 1 条数据
爬取了第 602 个主播共 1 条数据
爬取了第 603 个主播共 1 条数据
爬取了第 605 个主播共 44 条数据
爬取了第 607 个主播共 1 条数据
爬取了第 608 个主播共 125 条数据
爬取了第 609 个主播共 1 条数据
爬取了第 610 个主播共 12 条数据
爬取了第 611 个主播共 28 条数据
爬取了第 612 个主播共 70 条数据
爬取了第 614 个主播共 1 条数据
爬取了第 615 个主播共 86 条数据
爬取了第 616 个主播共 2 条数据
爬取了第 617 个主播共 33 条数据
爬取了第 618 个主播共 113 条数据
爬取了第 619 个主播共 1 条数据
爬取了第 620 个主播共 5 条数据
爬取了第 621 个主播共 1 条数据
爬取了第 622 个主播共 1 条数据
爬取了第 624 个主播共 82 条数据
爬取了第 626 个主播共 1 条数据
爬取了第 627 个主播共 1 条数据
爬取了第 628 个主播共 1 条数据
爬取

爬取了第 1067 个主播共 14 条数据
爬取了第 1069 个主播共 123 条数据
爬取了第 1070 个主播共 1 条数据
爬取了第 1072 个主播共 1 条数据
爬取了第 1073 个主播共 1 条数据
爬取了第 1074 个主播共 1 条数据
爬取了第 1077 个主播共 23 条数据
爬取了第 1078 个主播共 1 条数据
爬取了第 1079 个主播共 63 条数据
爬取了第 1080 个主播共 8 条数据
爬取了第 1081 个主播共 1 条数据
爬取了第 1082 个主播共 41 条数据
爬取了第 1083 个主播共 1 条数据
爬取了第 1084 个主播共 1 条数据
爬取了第 1086 个主播共 65 条数据
爬取了第 1087 个主播共 68 条数据
爬取了第 1088 个主播共 7 条数据
爬取了第 1089 个主播共 1 条数据
爬取了第 1090 个主播共 1 条数据
爬取了第 1092 个主播共 1 条数据
爬取了第 1095 个主播共 1 条数据
爬取了第 1096 个主播共 1 条数据
爬取了第 1097 个主播共 1 条数据
爬取了第 1098 个主播共 86 条数据
爬取了第 1099 个主播共 1 条数据
爬取了第 1100 个主播共 1 条数据
爬取了第 1103 个主播共 1 条数据
爬取了第 1104 个主播共 2 条数据
爬取了第 1105 个主播共 110 条数据
爬取了第 1106 个主播共 12 条数据
爬取了第 1107 个主播共 117 条数据
爬取了第 1108 个主播共 5 条数据
爬取了第 1109 个主播共 88 条数据
爬取了第 1110 个主播共 1 条数据
爬取了第 1111 个主播共 1 条数据
爬取了第 1112 个主播共 3 条数据
爬取了第 1113 个主播共 72 条数据
爬取了第 1114 个主播共 4 条数据
爬取了第 1115 个主播共 100 条数据
爬取了第 1116 个主播共 5 条数据
爬取了第 1117 个主播共 1 条数据
爬取了第 1118 个主播共 2 条数据
爬取了第 1119 个主播共 3 条数据
爬取了第 1120 个主播共 1 条数据
爬取了第 1121 个主播共 1 条数据
爬取了第 1122 个主播共 2 条数据
爬取了第 1123 个主播共 1

爬取了第 1534 个主播共 8 条数据
爬取了第 1535 个主播共 1 条数据
爬取了第 1536 个主播共 2 条数据
爬取了第 1537 个主播共 11 条数据
爬取了第 1539 个主播共 3 条数据
爬取了第 1540 个主播共 125 条数据
爬取了第 1542 个主播共 9 条数据
爬取了第 1543 个主播共 2 条数据
爬取了第 1544 个主播共 1 条数据
爬取了第 1546 个主播共 1 条数据
爬取了第 1548 个主播共 1 条数据
爬取了第 1549 个主播共 1 条数据
爬取了第 1550 个主播共 1 条数据
爬取了第 1551 个主播共 83 条数据
爬取了第 1552 个主播共 1 条数据
爬取了第 1553 个主播共 88 条数据
爬取了第 1554 个主播共 1 条数据
爬取了第 1555 个主播共 1 条数据
爬取了第 1556 个主播共 1 条数据
爬取了第 1557 个主播共 7 条数据
爬取了第 1558 个主播共 35 条数据
爬取了第 1561 个主播共 4 条数据
爬取了第 1562 个主播共 1 条数据
爬取了第 1563 个主播共 2 条数据
爬取了第 1564 个主播共 1 条数据
爬取了第 1565 个主播共 13 条数据
爬取了第 1566 个主播共 4 条数据
爬取了第 1568 个主播共 59 条数据
爬取了第 1569 个主播共 1 条数据
爬取了第 1571 个主播共 1 条数据
爬取了第 1572 个主播共 113 条数据
爬取了第 1573 个主播共 8 条数据
爬取了第 1574 个主播共 86 条数据
爬取了第 1576 个主播共 3 条数据
爬取了第 1577 个主播共 2 条数据
爬取了第 1578 个主播共 1 条数据
爬取了第 1579 个主播共 46 条数据
爬取了第 1581 个主播共 1 条数据
爬取了第 1583 个主播共 2 条数据
爬取了第 1584 个主播共 1 条数据
爬取了第 1585 个主播共 1 条数据
爬取了第 1586 个主播共 82 条数据
爬取了第 1587 个主播共 1 条数据
爬取了第 1588 个主播共 1 条数据
爬取了第 1589 个主播共 1 条数据
爬取了第 1590 个主播共 1 条数据
爬取了第 1591 个主播共 5 条数据


爬取了第 1993 个主播共 1 条数据
爬取了第 1994 个主播共 4 条数据
爬取了第 1995 个主播共 1 条数据
爬取了第 1997 个主播共 1 条数据
爬取了第 1998 个主播共 128 条数据
爬取了第 2000 个主播共 1 条数据
爬取了第 2001 个主播共 123 条数据
爬取了第 2003 个主播共 99 条数据
爬取了第 2004 个主播共 129 条数据
爬取了第 2005 个主播共 1 条数据
爬取了第 2009 个主播共 1 条数据
爬取了第 2011 个主播共 1 条数据
爬取了第 2012 个主播共 46 条数据
爬取了第 2013 个主播共 1 条数据
爬取了第 2014 个主播共 99 条数据
爬取了第 2015 个主播共 1 条数据
爬取了第 2016 个主播共 1 条数据
爬取了第 2017 个主播共 2 条数据
爬取了第 2018 个主播共 1 条数据
爬取了第 2019 个主播共 1 条数据
爬取了第 2020 个主播共 1 条数据
爬取了第 2021 个主播共 1 条数据
爬取了第 2023 个主播共 3 条数据
爬取了第 2024 个主播共 3 条数据
爬取了第 2027 个主播共 82 条数据
爬取了第 2028 个主播共 40 条数据
爬取了第 2029 个主播共 1 条数据
爬取了第 2030 个主播共 1 条数据
爬取了第 2031 个主播共 4 条数据
爬取了第 2032 个主播共 80 条数据
爬取了第 2033 个主播共 1 条数据
爬取了第 2034 个主播共 1 条数据
爬取了第 2035 个主播共 1 条数据
爬取了第 2037 个主播共 110 条数据
爬取了第 2038 个主播共 8 条数据
爬取了第 2039 个主播共 1 条数据
爬取了第 2040 个主播共 31 条数据
爬取了第 2041 个主播共 1 条数据
爬取了第 2043 个主播共 108 条数据
爬取了第 2044 个主播共 131 条数据
爬取了第 2045 个主播共 8 条数据
爬取了第 2046 个主播共 1 条数据
爬取了第 2047 个主播共 138 条数据
爬取了第 2048 个主播共 2 条数据
爬取了第 2049 个主播共 1 条数据
爬取了第 2050 个主播共 1 条数据
爬取了第 2051 个主播

...

爬取了第 3835 个主播共 69 条数据
爬取了第 3837 个主播共 1 条数据
爬取了第 3839 个主播共 1 条数据
爬取了第 3840 个主播共 126 条数据
爬取了第 3846 个主播共 109 条数据
爬取了第 3848 个主播共 26 条数据
爬取了第 3849 个主播共 139 条数据
爬取了第 3851 个主播共 8 条数据
爬取了第 3853 个主播共 135 条数据
爬取了第 3854 个主播共 95 条数据
爬取了第 3855 个主播共 130 条数据
爬取了第 3857 个主播共 99 条数据
爬取了第 3858 个主播共 102 条数据
爬取了第 3860 个主播共 114 条数据
爬取了第 3864 个主播共 113 条数据
爬取了第 3866 个主播共 1 条数据
爬取了第 3867 个主播共 2 条数据
爬取了第 3868 个主播共 105 条数据
爬取了第 3873 个主播共 10 条数据
爬取了第 3874 个主播共 34 条数据
爬取了第 3876 个主播共 58 条数据
爬取了第 3877 个主播共 66 条数据
爬取了第 3878 个主播共 1 条数据
爬取了第 3879 个主播共 138 条数据
爬取了第 3881 个主播共 132 条数据
爬取了第 3883 个主播共 33 条数据
爬取了第 3884 个主播共 119 条数据
爬取了第 3885 个主播共 132 条数据
爬取了第 3886 个主播共 132 条数据
爬取了第 3889 个主播共 50 条数据
爬取了第 3892 个主播共 2 条数据
爬取了第 3893 个主播共 1 条数据
爬取了第 3894 个主播共 5 条数据
爬取了第 3895 个主播共 120 条数据
爬取了第 3896 个主播共 112 条数据
爬取了第 3903 个主播共 5 条数据
爬取了第 3904 个主播共 32 条数据
爬取了第 3906 个主播共 48 条数据
爬取了第 3907 个主播共 6 条数据
爬取了第 3908 个主播共 11 条数据
爬取了第 3909 个主播共 111 条数据
爬取了第 3910 个主播共 1 条数据
爬取了第 3911 个主播共 14 条数据
爬取了第 3912 个主播共 86 条数据
爬取了第 3913 个主播共 60 条数据
爬取了第 

爬取了第 4323 个主播共 119 条数据
爬取了第 4324 个主播共 42 条数据
爬取了第 4325 个主播共 21 条数据
爬取了第 4326 个主播共 3 条数据
爬取了第 4328 个主播共 98 条数据
爬取了第 4329 个主播共 45 条数据
爬取了第 4330 个主播共 43 条数据
爬取了第 4331 个主播共 1 条数据
爬取了第 4332 个主播共 100 条数据
爬取了第 4333 个主播共 86 条数据
爬取了第 4334 个主播共 106 条数据
爬取了第 4335 个主播共 1 条数据
爬取了第 4336 个主播共 62 条数据
爬取了第 4337 个主播共 123 条数据
爬取了第 4338 个主播共 124 条数据
爬取了第 4339 个主播共 121 条数据
爬取了第 4340 个主播共 2 条数据
爬取了第 4341 个主播共 94 条数据
爬取了第 4342 个主播共 5 条数据
爬取了第 4343 个主播共 96 条数据
爬取了第 4344 个主播共 132 条数据
爬取了第 4345 个主播共 38 条数据
爬取了第 4346 个主播共 6 条数据
爬取了第 4347 个主播共 6 条数据
爬取了第 4348 个主播共 17 条数据
爬取了第 4350 个主播共 9 条数据
爬取了第 4351 个主播共 1 条数据
爬取了第 4352 个主播共 124 条数据
爬取了第 4353 个主播共 1 条数据
爬取了第 4354 个主播共 98 条数据
爬取了第 4355 个主播共 85 条数据
爬取了第 4356 个主播共 1 条数据
爬取了第 4357 个主播共 1 条数据
爬取了第 4358 个主播共 6 条数据
爬取了第 4359 个主播共 1 条数据
爬取了第 4360 个主播共 39 条数据
爬取了第 4361 个主播共 1 条数据
爬取了第 4362 个主播共 72 条数据
爬取了第 4363 个主播共 135 条数据
爬取了第 4366 个主播共 1 条数据
爬取了第 4370 个主播共 81 条数据
爬取了第 4373 个主播共 42 条数据
爬取了第 4374 个主播共 116 条数据
爬取了第 4375 个主播共 111 条数据
爬取了第 4376 个主播共 100 条数据
爬取了第 4378 个主播共

爬取了第 5579 个主播共 27 条数据
爬取了第 5584 个主播共 49 条数据
爬取了第 5612 个主播共 54 条数据
爬取了第 5614 个主播共 43 条数据
爬取了第 5616 个主播共 100 条数据
爬取了第 5619 个主播共 30 条数据
爬取了第 5620 个主播共 2 条数据
爬取了第 5629 个主播共 76 条数据
爬取了第 5630 个主播共 109 条数据
爬取了第 5631 个主播共 138 条数据
爬取了第 5633 个主播共 34 条数据
爬取了第 5641 个主播共 1 条数据
爬取了第 5642 个主播共 18 条数据
爬取了第 5650 个主播共 53 条数据
爬取了第 5660 个主播共 2 条数据
爬取了第 5665 个主播共 99 条数据
爬取了第 5667 个主播共 5 条数据
爬取了第 5672 个主播共 32 条数据
爬取了第 5695 个主播共 48 条数据
爬取了第 5702 个主播共 53 条数据
爬取了第 5711 个主播共 13 条数据
爬取了第 5712 个主播共 67 条数据
爬取了第 5714 个主播共 11 条数据
爬取了第 5715 个主播共 111 条数据
爬取了第 5717 个主播共 2 条数据
爬取了第 5718 个主播共 77 条数据
爬取了第 5719 个主播共 98 条数据
爬取了第 5725 个主播共 11 条数据
爬取了第 5726 个主播共 114 条数据
爬取了第 5727 个主播共 26 条数据
爬取了第 5730 个主播共 103 条数据
爬取了第 5733 个主播共 135 条数据
爬取了第 5739 个主播共 19 条数据
爬取了第 5740 个主播共 1 条数据
爬取了第 5741 个主播共 95 条数据
爬取了第 5750 个主播共 73 条数据
爬取了第 5752 个主播共 3 条数据
爬取了第 5754 个主播共 1 条数据
爬取了第 5769 个主播共 121 条数据
爬取了第 5776 个主播共 3 条数据
爬取了第 5783 个主播共 19 条数据
爬取了第 5800 个主播共 1 条数据
爬取了第 5810 个主播共 117 条数据
爬取了第 5828 个主播共 117 条数据
爬取了第 5842 个主播共 101 条数据
爬取了第 5870

爬取了第 8474 个主播共 77 条数据
爬取了第 8486 个主播共 87 条数据
爬取了第 8497 个主播共 132 条数据
爬取了第 8533 个主播共 1 条数据
爬取了第 8543 个主播共 98 条数据
爬取了第 8546 个主播共 1 条数据
爬取了第 8548 个主播共 4 条数据
爬取了第 8565 个主播共 115 条数据
爬取了第 8574 个主播共 1 条数据
爬取了第 8590 个主播共 43 条数据
爬取了第 8596 个主播共 122 条数据
爬取了第 8598 个主播共 68 条数据
爬取了第 8600 个主播共 117 条数据
爬取了第 8602 个主播共 103 条数据
爬取了第 8604 个主播共 132 条数据
爬取了第 8605 个主播共 2 条数据
爬取了第 8608 个主播共 52 条数据
爬取了第 8611 个主播共 91 条数据
爬取了第 8615 个主播共 3 条数据
爬取了第 8625 个主播共 9 条数据
爬取了第 8637 个主播共 136 条数据
爬取了第 8643 个主播共 130 条数据
爬取了第 8651 个主播共 117 条数据
爬取了第 8672 个主播共 98 条数据
爬取了第 8673 个主播共 26 条数据
爬取了第 8686 个主播共 71 条数据
爬取了第 8695 个主播共 44 条数据
爬取了第 8704 个主播共 56 条数据
爬取了第 8705 个主播共 123 条数据
爬取了第 8713 个主播共 134 条数据
爬取了第 8714 个主播共 131 条数据
爬取了第 8723 个主播共 102 条数据
爬取了第 8728 个主播共 2 条数据
爬取了第 8743 个主播共 89 条数据
爬取了第 8778 个主播共 1 条数据
爬取了第 8780 个主播共 130 条数据
爬取了第 8781 个主播共 2 条数据
爬取了第 8785 个主播共 45 条数据
爬取了第 8798 个主播共 8 条数据
爬取了第 8829 个主播共 7 条数据
爬取了第 8834 个主播共 68 条数据
爬取了第 8837 个主播共 119 条数据
爬取了第 8838 个主播共 70 条数据
爬取了第 8844 个主播共 140 条数据
爬取了第 8847 个主播共 8 条数据
爬取了第 88

爬取了第 11154 个主播共 85 条数据
爬取了第 11186 个主播共 96 条数据
爬取了第 11190 个主播共 113 条数据
爬取了第 11195 个主播共 104 条数据
爬取了第 11203 个主播共 43 条数据
爬取了第 11214 个主播共 44 条数据
爬取了第 11221 个主播共 1 条数据
爬取了第 11225 个主播共 139 条数据
爬取了第 11230 个主播共 92 条数据
爬取了第 11236 个主播共 2 条数据
爬取了第 11243 个主播共 133 条数据
爬取了第 11245 个主播共 112 条数据
爬取了第 11247 个主播共 100 条数据
爬取了第 11274 个主播共 128 条数据
爬取了第 11282 个主播共 113 条数据
爬取了第 11283 个主播共 132 条数据
爬取了第 11329 个主播共 81 条数据
爬取了第 11340 个主播共 1 条数据
爬取了第 11351 个主播共 6 条数据
爬取了第 11360 个主播共 53 条数据
爬取了第 11363 个主播共 41 条数据
爬取了第 11365 个主播共 118 条数据
爬取了第 11366 个主播共 1 条数据
爬取了第 11368 个主播共 117 条数据
爬取了第 11375 个主播共 33 条数据
爬取了第 11377 个主播共 131 条数据
爬取了第 11378 个主播共 131 条数据
爬取了第 11388 个主播共 16 条数据
爬取了第 11396 个主播共 1 条数据
爬取了第 11410 个主播共 8 条数据
爬取了第 11413 个主播共 2 条数据
爬取了第 11420 个主播共 70 条数据
爬取了第 11435 个主播共 9 条数据
爬取了第 11438 个主播共 41 条数据
爬取了第 11445 个主播共 97 条数据
爬取了第 11480 个主播共 8 条数据
爬取了第 11481 个主播共 114 条数据
爬取了第 11483 个主播共 115 条数据
爬取了第 11490 个主播共 36 条数据
爬取了第 11499 个主播共 125 条数据
爬取了第 11506 个主播共 40 条数据
爬取了第 11516 个主播共 21 条数据
爬取了第 11518 个主播共 25 条数据
爬取了第 

In [3]:
#有直播数据的mid
df=pd.read_csv("Crawled_Dataset\直播间营收数据.csv",encoding="UTF-8")
data=df[df['count'] != 0]
mid=data['mid'].unique()

10196457

爬取粉丝数量数据

Crawling the number of followers data

In [84]:
df=pd.DataFrame()
url1='https://api.zeroroku.com/bilibili/author/fans?mid=672328094'
r_fan=requests.get(url1,headers=headers)
fan_data=json.loads(r_fan.text)

if datas!={"message":"live room not found"}:#部分主播没有收录
mid_num=[mid[i]]*len(fan_data)
df_fan=DataFrame(fan_data)
df_fan.insert(0,'mid',mid_num)
df=df.append(df_fan,ignore_index=True)
df

,mid,fans,created_at,rate1,rate7
0,349308374221068,41737,2021-02-04T15:00:44.219Z,NaN,NaN
1,349308374221068,43368,2021-02-05T15:08:02.948Z,NaN,NaN
2,349308374221068,45534,2021-02-06T15:16:01.61Z,NaN,NaN
3,349308374221068,48331,2021-02-07T15:24:23.215Z,NaN,NaN
4,349308374221068,53103,2021-02-08T15:29:18.364Z,NaN,NaN
...,...,...,...,...,...
1325,349308374221068,1726030,2023-01-15T13:31:30.164318Z,3942.0,17309.0
1326,349308374221068,1728540,2023-01-16T14:00:54.517824Z,2459.0,19770.0
1327,349308374221068,1730371,2023-01-17T14:54:19.72308Z,1765.0,21352.0
1328,349308374221068,1731473,2023-01-18T14:21:18.154935Z,1140.0,22160.0


In [105]:
df=pd.DataFrame()

#单个主播粉丝数
for i in range(0,len(mid)):
    url1='https://api.zeroroku.com/bilibili/author/fans?mid='+str(mid[i])
    r_fan=requests.get(url1,headers=headers)
    fan_data=json.loads(r_fan.text)
    
    mid_num=[mid[i]]*len(fan_data)
    df_fan=DataFrame(fan_data)
    df_fan.insert(0,'mid',mid_num)
    df=df.append(df_fan,ignore_index=True)
    print("爬取了第",i,"个主播共",len(fan_data),"条数据")

#写入excel
print('开始存储')
df.to_csv('Crawled_Dataset\虚拟主播粉丝数据.csv',mode='w',encoding='gb18030')
print('存储完毕')        

爬取了第 0 个主播共 1302 条数据
爬取了第 1 个主播共 1054 条数据
爬取了第 2 个主播共 911 条数据
爬取了第 3 个主播共 993 条数据
爬取了第 4 个主播共 323 条数据
爬取了第 5 个主播共 914 条数据
爬取了第 6 个主播共 256 条数据
爬取了第 7 个主播共 605 条数据
爬取了第 8 个主播共 941 条数据
爬取了第 9 个主播共 974 条数据
爬取了第 10 个主播共 956 条数据
爬取了第 11 个主播共 864 条数据
爬取了第 12 个主播共 579 条数据
爬取了第 13 个主播共 510 条数据
爬取了第 14 个主播共 431 条数据
爬取了第 15 个主播共 1073 条数据
爬取了第 16 个主播共 578 条数据
爬取了第 17 个主播共 513 条数据
爬取了第 18 个主播共 470 条数据
爬取了第 19 个主播共 779 条数据
爬取了第 20 个主播共 354 条数据
爬取了第 21 个主播共 532 条数据
爬取了第 22 个主播共 464 条数据
爬取了第 23 个主播共 983 条数据
爬取了第 24 个主播共 520 条数据
爬取了第 25 个主播共 472 条数据
爬取了第 26 个主播共 876 条数据
爬取了第 27 个主播共 504 条数据
爬取了第 28 个主播共 491 条数据
爬取了第 29 个主播共 1311 条数据
爬取了第 30 个主播共 1042 条数据
爬取了第 31 个主播共 1142 条数据
爬取了第 32 个主播共 247 条数据
爬取了第 33 个主播共 1168 条数据
爬取了第 34 个主播共 433 条数据
爬取了第 35 个主播共 737 条数据
爬取了第 36 个主播共 1282 条数据
爬取了第 37 个主播共 659 条数据
爬取了第 38 个主播共 521 条数据
爬取了第 39 个主播共 400 条数据
爬取了第 40 个主播共 1089 条数据
爬取了第 41 个主播共 367 条数据
爬取了第 42 个主播共 351 条数据
爬取了第 43 个主播共 999 条数据
爬取了第 44 个主播共 1255 条数据
爬取了第 45 个主播共 1425 条数据
爬取了第 46 个主播共 983 条数据
爬取了第 47 个主播共

爬取了第 374 个主播共 971 条数据
爬取了第 375 个主播共 225 条数据
爬取了第 376 个主播共 481 条数据
爬取了第 377 个主播共 289 条数据
爬取了第 378 个主播共 1026 条数据
爬取了第 379 个主播共 707 条数据
爬取了第 380 个主播共 517 条数据
爬取了第 381 个主播共 616 条数据
爬取了第 382 个主播共 499 条数据
爬取了第 383 个主播共 847 条数据
爬取了第 384 个主播共 546 条数据
爬取了第 385 个主播共 647 条数据
爬取了第 386 个主播共 1187 条数据
爬取了第 387 个主播共 409 条数据
爬取了第 388 个主播共 940 条数据
爬取了第 389 个主播共 441 条数据
爬取了第 390 个主播共 785 条数据
爬取了第 391 个主播共 629 条数据
爬取了第 392 个主播共 1120 条数据
爬取了第 393 个主播共 1022 条数据
爬取了第 394 个主播共 934 条数据
爬取了第 395 个主播共 785 条数据
爬取了第 396 个主播共 1007 条数据
爬取了第 397 个主播共 494 条数据
爬取了第 398 个主播共 963 条数据
爬取了第 399 个主播共 1090 条数据
爬取了第 400 个主播共 503 条数据
爬取了第 401 个主播共 1435 条数据
爬取了第 402 个主播共 528 条数据
爬取了第 403 个主播共 1264 条数据
爬取了第 404 个主播共 915 条数据
爬取了第 405 个主播共 535 条数据
爬取了第 406 个主播共 579 条数据
爬取了第 407 个主播共 857 条数据
爬取了第 408 个主播共 618 条数据
爬取了第 409 个主播共 494 条数据
爬取了第 410 个主播共 1372 条数据
爬取了第 411 个主播共 1130 条数据
爬取了第 412 个主播共 404 条数据
爬取了第 413 个主播共 1163 条数据
爬取了第 414 个主播共 551 条数据
爬取了第 415 个主播共 601 条数据
爬取了第 416 个主播共 805 条数据
爬取了第 417 个主播共 1138 条数据
爬取了第 418 个主播共 598 条数

爬取了第 744 个主播共 1024 条数据
爬取了第 745 个主播共 443 条数据
爬取了第 746 个主播共 984 条数据
爬取了第 747 个主播共 989 条数据
爬取了第 748 个主播共 1110 条数据
爬取了第 749 个主播共 288 条数据
爬取了第 750 个主播共 426 条数据
爬取了第 751 个主播共 539 条数据
爬取了第 752 个主播共 1168 条数据
爬取了第 753 个主播共 458 条数据
爬取了第 754 个主播共 443 条数据
爬取了第 755 个主播共 622 条数据
爬取了第 756 个主播共 310 条数据
爬取了第 757 个主播共 560 条数据
爬取了第 758 个主播共 1117 条数据
爬取了第 759 个主播共 1137 条数据
爬取了第 760 个主播共 1272 条数据
爬取了第 761 个主播共 1046 条数据
爬取了第 762 个主播共 458 条数据
爬取了第 763 个主播共 915 条数据
爬取了第 764 个主播共 240 条数据
爬取了第 765 个主播共 974 条数据
爬取了第 766 个主播共 1145 条数据
爬取了第 767 个主播共 1196 条数据
爬取了第 768 个主播共 950 条数据
爬取了第 769 个主播共 498 条数据
爬取了第 770 个主播共 521 条数据
爬取了第 771 个主播共 347 条数据
爬取了第 772 个主播共 444 条数据
爬取了第 773 个主播共 906 条数据
爬取了第 774 个主播共 506 条数据
爬取了第 775 个主播共 874 条数据
爬取了第 776 个主播共 318 条数据
爬取了第 777 个主播共 278 条数据
爬取了第 778 个主播共 535 条数据
爬取了第 779 个主播共 432 条数据
爬取了第 780 个主播共 1104 条数据
爬取了第 781 个主播共 559 条数据
爬取了第 782 个主播共 66 条数据
爬取了第 783 个主播共 1246 条数据
爬取了第 784 个主播共 1132 条数据
爬取了第 785 个主播共 253 条数据
爬取了第 786 个主播共 25 条数据
爬取了第 787 个主播共 318 条数据
爬取了第 788 个主播共 778 条数据


爬取了第 1110 个主播共 232 条数据
爬取了第 1111 个主播共 1116 条数据
爬取了第 1112 个主播共 362 条数据
爬取了第 1113 个主播共 475 条数据
爬取了第 1114 个主播共 520 条数据
爬取了第 1115 个主播共 419 条数据
爬取了第 1116 个主播共 207 条数据
爬取了第 1117 个主播共 313 条数据
爬取了第 1118 个主播共 267 条数据
爬取了第 1119 个主播共 412 条数据
爬取了第 1120 个主播共 550 条数据
爬取了第 1121 个主播共 630 条数据
爬取了第 1122 个主播共 400 条数据
爬取了第 1123 个主播共 468 条数据
爬取了第 1124 个主播共 439 条数据
爬取了第 1125 个主播共 390 条数据
爬取了第 1126 个主播共 290 条数据
爬取了第 1127 个主播共 615 条数据
爬取了第 1128 个主播共 348 条数据
爬取了第 1129 个主播共 1485 条数据
爬取了第 1130 个主播共 320 条数据
爬取了第 1131 个主播共 178 条数据
爬取了第 1132 个主播共 384 条数据
爬取了第 1133 个主播共 272 条数据
爬取了第 1134 个主播共 254 条数据
爬取了第 1135 个主播共 375 条数据
爬取了第 1136 个主播共 229 条数据
爬取了第 1137 个主播共 581 条数据
爬取了第 1138 个主播共 595 条数据
爬取了第 1139 个主播共 373 条数据
爬取了第 1140 个主播共 356 条数据
爬取了第 1141 个主播共 279 条数据
爬取了第 1142 个主播共 608 条数据
爬取了第 1143 个主播共 335 条数据
爬取了第 1144 个主播共 252 条数据
爬取了第 1145 个主播共 408 条数据
爬取了第 1146 个主播共 618 条数据
爬取了第 1147 个主播共 458 条数据
爬取了第 1148 个主播共 319 条数据
爬取了第 1149 个主播共 451 条数据
爬取了第 1150 个主播共 1356 条数据
爬取了第 1151 个主播共 422 条数据
爬取了第 1152 个主播共 640 条数据
爬取了第 115

爬取了第 1466 个主播共 1292 条数据
爬取了第 1467 个主播共 521 条数据
爬取了第 1468 个主播共 222 条数据
爬取了第 1469 个主播共 88 条数据
爬取了第 1470 个主播共 402 条数据
爬取了第 1471 个主播共 251 条数据
爬取了第 1472 个主播共 936 条数据
爬取了第 1473 个主播共 483 条数据
爬取了第 1474 个主播共 384 条数据
爬取了第 1475 个主播共 420 条数据
爬取了第 1476 个主播共 360 条数据
爬取了第 1477 个主播共 961 条数据
爬取了第 1478 个主播共 584 条数据
爬取了第 1479 个主播共 962 条数据
爬取了第 1480 个主播共 336 条数据
爬取了第 1481 个主播共 910 条数据
爬取了第 1482 个主播共 508 条数据
爬取了第 1483 个主播共 1281 条数据
爬取了第 1484 个主播共 265 条数据
爬取了第 1485 个主播共 251 条数据
爬取了第 1486 个主播共 208 条数据
爬取了第 1487 个主播共 488 条数据
爬取了第 1488 个主播共 220 条数据
爬取了第 1489 个主播共 1375 条数据
爬取了第 1490 个主播共 447 条数据
爬取了第 1491 个主播共 380 条数据
爬取了第 1492 个主播共 915 条数据
爬取了第 1493 个主播共 418 条数据
爬取了第 1494 个主播共 256 条数据
爬取了第 1495 个主播共 219 条数据
爬取了第 1496 个主播共 440 条数据
爬取了第 1497 个主播共 427 条数据
爬取了第 1498 个主播共 381 条数据
爬取了第 1499 个主播共 1291 条数据
爬取了第 1500 个主播共 415 条数据
爬取了第 1501 个主播共 473 条数据
爬取了第 1502 个主播共 1241 条数据
爬取了第 1503 个主播共 253 条数据
爬取了第 1504 个主播共 262 条数据
爬取了第 1505 个主播共 490 条数据
爬取了第 1506 个主播共 245 条数据
爬取了第 1507 个主播共 1383 条数据
爬取了第 1508 个主播共 424 条数据
爬取了第 1

爬取了第 1822 个主播共 289 条数据
爬取了第 1823 个主播共 255 条数据
爬取了第 1824 个主播共 397 条数据
爬取了第 1825 个主播共 286 条数据
爬取了第 1826 个主播共 304 条数据
爬取了第 1827 个主播共 310 条数据
爬取了第 1828 个主播共 1179 条数据
爬取了第 1829 个主播共 257 条数据
爬取了第 1830 个主播共 254 条数据
爬取了第 1831 个主播共 180 条数据
爬取了第 1832 个主播共 175 条数据
爬取了第 1833 个主播共 205 条数据
爬取了第 1834 个主播共 1367 条数据
爬取了第 1835 个主播共 280 条数据
爬取了第 1836 个主播共 220 条数据
爬取了第 1837 个主播共 227 条数据
爬取了第 1838 个主播共 304 条数据
爬取了第 1839 个主播共 776 条数据
爬取了第 1840 个主播共 342 条数据
爬取了第 1841 个主播共 246 条数据
爬取了第 1842 个主播共 249 条数据
爬取了第 1843 个主播共 1142 条数据
爬取了第 1844 个主播共 249 条数据
爬取了第 1845 个主播共 199 条数据
爬取了第 1846 个主播共 251 条数据
爬取了第 1847 个主播共 277 条数据
爬取了第 1848 个主播共 168 条数据
爬取了第 1849 个主播共 220 条数据
爬取了第 1850 个主播共 385 条数据
爬取了第 1851 个主播共 285 条数据
爬取了第 1852 个主播共 334 条数据
爬取了第 1853 个主播共 353 条数据
爬取了第 1854 个主播共 255 条数据
爬取了第 1855 个主播共 252 条数据
爬取了第 1856 个主播共 239 条数据
爬取了第 1857 个主播共 1397 条数据
爬取了第 1858 个主播共 270 条数据
爬取了第 1859 个主播共 252 条数据
爬取了第 1860 个主播共 250 条数据
爬取了第 1861 个主播共 283 条数据
爬取了第 1862 个主播共 253 条数据
爬取了第 1863 个主播共 194 条数据
爬取了第 1864 个主播共 274 条数据
爬取了第 18

爬取了第 2178 个主播共 301 条数据
爬取了第 2179 个主播共 878 条数据
爬取了第 2180 个主播共 252 条数据
爬取了第 2181 个主播共 226 条数据
爬取了第 2182 个主播共 203 条数据
爬取了第 2183 个主播共 874 条数据
爬取了第 2184 个主播共 334 条数据
爬取了第 2185 个主播共 644 条数据
爬取了第 2186 个主播共 67 条数据
爬取了第 2187 个主播共 208 条数据
爬取了第 2188 个主播共 247 条数据
爬取了第 2189 个主播共 237 条数据
爬取了第 2190 个主播共 418 条数据
爬取了第 2191 个主播共 177 条数据
爬取了第 2192 个主播共 179 条数据
爬取了第 2193 个主播共 257 条数据
爬取了第 2194 个主播共 169 条数据
爬取了第 2195 个主播共 139 条数据
爬取了第 2196 个主播共 244 条数据
爬取了第 2197 个主播共 188 条数据
爬取了第 2198 个主播共 1108 条数据
爬取了第 2199 个主播共 195 条数据
爬取了第 2200 个主播共 161 条数据
爬取了第 2201 个主播共 168 条数据
爬取了第 2202 个主播共 140 条数据
爬取了第 2203 个主播共 187 条数据
爬取了第 2204 个主播共 239 条数据
爬取了第 2205 个主播共 257 条数据
爬取了第 2206 个主播共 241 条数据
爬取了第 2207 个主播共 198 条数据
爬取了第 2208 个主播共 156 条数据
爬取了第 2209 个主播共 911 条数据
爬取了第 2210 个主播共 246 条数据
爬取了第 2211 个主播共 390 条数据
爬取了第 2212 个主播共 238 条数据
爬取了第 2213 个主播共 42 条数据
爬取了第 2214 个主播共 153 条数据
爬取了第 2215 个主播共 248 条数据
爬取了第 2216 个主播共 247 条数据
爬取了第 2217 个主播共 314 条数据
爬取了第 2218 个主播共 254 条数据
爬取了第 2219 个主播共 249 条数据
爬取了第 2220 个主播共 146 条数据
爬取了第 2221 个主

爬取了第 2536 个主播共 241 条数据
爬取了第 2537 个主播共 197 条数据
爬取了第 2538 个主播共 61 条数据
爬取了第 2539 个主播共 188 条数据
爬取了第 2540 个主播共 245 条数据
爬取了第 2541 个主播共 163 条数据
爬取了第 2542 个主播共 145 条数据
爬取了第 2543 个主播共 175 条数据
爬取了第 2544 个主播共 231 条数据
爬取了第 2545 个主播共 1418 条数据
爬取了第 2546 个主播共 234 条数据
爬取了第 2547 个主播共 255 条数据
爬取了第 2548 个主播共 195 条数据
爬取了第 2549 个主播共 182 条数据
爬取了第 2550 个主播共 214 条数据
爬取了第 2551 个主播共 218 条数据
爬取了第 2552 个主播共 416 条数据
爬取了第 2553 个主播共 192 条数据
爬取了第 2554 个主播共 150 条数据
爬取了第 2555 个主播共 299 条数据
爬取了第 2556 个主播共 210 条数据
爬取了第 2557 个主播共 225 条数据
爬取了第 2558 个主播共 249 条数据
爬取了第 2559 个主播共 143 条数据
爬取了第 2560 个主播共 237 条数据
爬取了第 2561 个主播共 145 条数据
爬取了第 2562 个主播共 146 条数据
爬取了第 2563 个主播共 179 条数据
爬取了第 2564 个主播共 154 条数据
爬取了第 2565 个主播共 238 条数据
爬取了第 2566 个主播共 236 条数据
爬取了第 2567 个主播共 131 条数据
爬取了第 2568 个主播共 166 条数据
爬取了第 2569 个主播共 108 条数据
爬取了第 2570 个主播共 222 条数据
爬取了第 2571 个主播共 1050 条数据
爬取了第 2572 个主播共 1246 条数据
爬取了第 2573 个主播共 123 条数据
爬取了第 2574 个主播共 252 条数据
爬取了第 2575 个主播共 63 条数据
爬取了第 2576 个主播共 415 条数据
爬取了第 2577 个主播共 92 条数据
爬取了第 2578 个主播共 114 条数据
爬取了第 2579 个

爬取了第 2894 个主播共 117 条数据
爬取了第 2895 个主播共 57 条数据
爬取了第 2896 个主播共 147 条数据
爬取了第 2897 个主播共 159 条数据
爬取了第 2898 个主播共 135 条数据
爬取了第 2899 个主播共 25 条数据
爬取了第 2900 个主播共 79 条数据
爬取了第 2901 个主播共 138 条数据
爬取了第 2902 个主播共 140 条数据
爬取了第 2903 个主播共 149 条数据
爬取了第 2904 个主播共 902 条数据
爬取了第 2905 个主播共 201 条数据
爬取了第 2906 个主播共 131 条数据
爬取了第 2907 个主播共 211 条数据
爬取了第 2908 个主播共 117 条数据
爬取了第 2909 个主播共 126 条数据
爬取了第 2910 个主播共 213 条数据
爬取了第 2911 个主播共 97 条数据
爬取了第 2912 个主播共 25 条数据
爬取了第 2913 个主播共 61 条数据
爬取了第 2914 个主播共 114 条数据
爬取了第 2915 个主播共 146 条数据
爬取了第 2916 个主播共 117 条数据
爬取了第 2917 个主播共 175 条数据
爬取了第 2918 个主播共 102 条数据
爬取了第 2919 个主播共 123 条数据
爬取了第 2920 个主播共 416 条数据
爬取了第 2921 个主播共 133 条数据
爬取了第 2922 个主播共 251 条数据
爬取了第 2923 个主播共 246 条数据
爬取了第 2924 个主播共 97 条数据
爬取了第 2925 个主播共 234 条数据
爬取了第 2926 个主播共 117 条数据
爬取了第 2927 个主播共 255 条数据
爬取了第 2928 个主播共 211 条数据
爬取了第 2929 个主播共 117 条数据
爬取了第 2930 个主播共 89 条数据
爬取了第 2931 个主播共 126 条数据
爬取了第 2932 个主播共 135 条数据
爬取了第 2933 个主播共 127 条数据
爬取了第 2934 个主播共 422 条数据
爬取了第 2935 个主播共 192 条数据
爬取了第 2936 个主播共 42 条数据
爬取了第 2937 个主播共 103 条

# bilibili.com

爬取主播发布的视频数据

Crawling video data posted by anchors

In [15]:
import requests    #导入requests包
import json
import pandas as pd
from bs4 import BeautifulSoup

headers={
    'Cookie': 'LIVE_BUVID=AUTO4915690662054650; PVID=2; i-wanna-go-back=-1; buvid4=2D49B44C-89BE-3459-4F3A-D693340D478841315-022030122-95aQASM5XNUcw9eXOr0sqg%3D%3D; CURRENT_BLACKGAP=0; buvid_fp_plain=undefined; DedeUserID=37159596; DedeUserID__ckMd5=90ff347065a1c2b3; b_ut=5; nostalgia_conf=-1; blackside_state=0; fingerprint3=47304c599a5eb446098accef88cd00b4; hit-dyn-v2=1; CURRENT_QUALITY=0; buvid3=579A148B-1519-CB4A-594F-4BC06CE0578A32551infoc; b_nut=1664688232; buvid_fp=579A148B-1519-CB4A-594F-4BC06CE0578A32551infoc; rpdid=|(u)~ll~m|um0J\'uYYmmlRkY|; _uuid=DDAD56DE-8F29-16F7-2BCB-8DE628749B6F87565infoc; is-2022-channel=1; CURRENT_FNVAL=4048; hit-new-style-dyn=0; bp_video_offset_37159596=753253533561651200; SESSDATA=d9ceec7e%2C1689782446%2Ca433b%2A12; bili_jct=4e47e74888bcd9273d4d878f1881d38f; sid=8f4yyurc; b_lsid=ACE597DB_185D213C42B; innersign=1; fingerprint=07b393547c242a1afaa9f88596a357c6',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.55',
    #'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    #'Connection': 'keep-alive'
}

In [20]:
video_total1=pd.DataFrame()
tag_total1=pd.DataFrame()

In [127]:
import time
video_total=pd.DataFrame()
tag_total=pd.DataFrame()
for j in range(3150,3171):
    url1='https://api.bilibili.com/x/space/wbi/arc/search?mid='+str(mid[j])+'&ps=30&tid=0&pn=1&keyword=&order=pubdate&order_avoided=true&w_rid=54bcec3e57e0e62dbdb0814ea6553c13&wts=1674271892'
    r_video=requests.get(url1,headers=headers)
    video=json.loads(r_video.text)

    import math
    video_num=video['data']['page']['count'] #视频总数
    pagenum=math.ceil(int(video_num)/30) #总共页数
    mid_num=[mid[j]]*video_num
    
    if video_num==0:
        tag_total=tag_total.append({'mid':mid[j],'video_num':0},ignore_index=True)
    else:
        #视频tag
        taglist=pd.DataFrame(video['data']['list']['tlist'])
        taglist.columns=taglist.loc['name']
        taglist=taglist[1:2]
        taglist.insert(0,'video_num',video_num)
        taglist.insert(0,'mid',mid[j])
        tag_total=tag_total.append(taglist,ignore_index=True)
        #taglist

        video_each=pd.DataFrame()
        for i in range(1,pagenum+1):
            #time.sleep(2)
            url='https://api.bilibili.com/x/space/wbi/arc/search?mid='+str(mid[j])+'&ps=30&tid=0&pn='+str(i)+'&keyword=&order=pubdate&order_avoided=true&w_rid=54bcec3e57e0e62dbdb0814ea6553c13&wts=1674271892'
            r_video=requests.get(url,headers=headers)
            video=json.loads(r_video.text)

            import datetime
            video_df=pd.DataFrame(video['data']['list']['vlist'])[['aid','created','typeid','comment','play','video_review','length','is_union_video','meta']]
            video_df['created']=video_df['created'].map(lambda x: datetime.datetime.fromtimestamp(int(x)))
            video_each=video_each.append(video_df)
            print('已爬取了第',j,'个主播第',i,'页视频数据')
        video_each.insert(0,'mid',mid_num)
        video_total=video_total.append(video_each,ignore_index=True)
    print('第',j,'个主播视频数据爬取完毕')
#写入excel
print('开始存储')
tag_total1=tag_total1.append(tag_total,ignore_index=True)
video_total1=video_total1.append(video_total,ignore_index=True)
print('存储完毕')

已爬取了第 3150 个主播第 1 页视频数据
已爬取了第 3150 个主播第 2 页视频数据
已爬取了第 3150 个主播第 3 页视频数据
已爬取了第 3150 个主播第 4 页视频数据
已爬取了第 3150 个主播第 5 页视频数据
已爬取了第 3150 个主播第 6 页视频数据
已爬取了第 3150 个主播第 7 页视频数据
第 3150 个主播视频数据爬取完毕
已爬取了第 3151 个主播第 1 页视频数据
第 3151 个主播视频数据爬取完毕
已爬取了第 3152 个主播第 1 页视频数据
已爬取了第 3152 个主播第 2 页视频数据
第 3152 个主播视频数据爬取完毕
已爬取了第 3153 个主播第 1 页视频数据
第 3153 个主播视频数据爬取完毕
已爬取了第 3154 个主播第 1 页视频数据
第 3154 个主播视频数据爬取完毕
已爬取了第 3155 个主播第 1 页视频数据
第 3155 个主播视频数据爬取完毕
已爬取了第 3156 个主播第 1 页视频数据
已爬取了第 3156 个主播第 2 页视频数据
已爬取了第 3156 个主播第 3 页视频数据
已爬取了第 3156 个主播第 4 页视频数据
已爬取了第 3156 个主播第 5 页视频数据
已爬取了第 3156 个主播第 6 页视频数据
已爬取了第 3156 个主播第 7 页视频数据
已爬取了第 3156 个主播第 8 页视频数据
已爬取了第 3156 个主播第 9 页视频数据
已爬取了第 3156 个主播第 10 页视频数据
已爬取了第 3156 个主播第 11 页视频数据
已爬取了第 3156 个主播第 12 页视频数据
已爬取了第 3156 个主播第 13 页视频数据
已爬取了第 3156 个主播第 14 页视频数据
已爬取了第 3156 个主播第 15 页视频数据
第 3156 个主播视频数据爬取完毕
已爬取了第 3157 个主播第 1 页视频数据
第 3157 个主播视频数据爬取完毕
已爬取了第 3158 个主播第 1 页视频数据
第 3158 个主播视频数据爬取完毕
已爬取了第 3159 个主播第 1 页视频数据
第 3159 个主播视频数据爬取完毕
已爬取了第 3160 个主播第 1 页视频数据
第 3160 个主播视频数据爬取完毕
已爬取了第 3161 个主播第 1

In [128]:
tag_total1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1571 entries, 0 to 1570
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   mid        1571 non-null   float64
 1   video_num  1571 non-null   float64
 2   动画         1328 non-null   object 
 3   音乐         1078 non-null   object 
 4   游戏         970 non-null    object 
 5   舞蹈         93 non-null     object 
 6   生活         965 non-null    object 
 7   动物圈        160 non-null    object 
 8   娱乐         85 non-null     object 
 9   影视         95 non-null     object 
 10  美食         96 non-null     object 
 11  知识         91 non-null     object 
 12  鬼畜         39 non-null     object 
 13  科技         43 non-null     object 
 14  国创         36 non-null     object 
 15  时尚         27 non-null     object 
 16  汽车         9 non-null      object 
 17  资讯         6 non-null      object 
 18  纪录片        9 non-null      object 
 19  运动         24 non-null     object 
 20  番剧      

In [129]:
tag_total1.to_csv('Crawled_Dataset\虚拟主播视频tag.csv',mode='a',encoding='gb18030')
video_total1.to_csv('Crawled_Dataset\虚拟主播视频数据.csv',mode='a',encoding='gb18030')

爬取大粉数据

In [118]:
mid1=[]
bigfan_num=[]
bigfan_total=[]

#单个主播大粉
for i in range(0,len(mid)):
    url1='https://api.zeroroku.com/bilibili/author/famous-fans?mid='+str(mid[i])
    r_bigfan=requests.get(url1,headers=headers)
    bigfan=json.loads(r_bigfan.text)
    
    if bigfan is None:
        bigfan_num.append('0')
        bigfan_total.append('0')
        mid1.append(str(mid[i]))
        print("第",i,"个主播无大粉")
    else:
        df_bigfan=DataFrame(bigfan)
        bigfan_num.append(len(bigfan))
        bigfan_total.append(df_bigfan['fans'].sum())
        mid1.append(str(mid[i]))
        print("第",i,"个主播共",len(bigfan),"个大粉")

#统一合并df
bigfan_dict={'mid': mid1,'bigfan_num': bigfan_num,'bigfan_total':bigfan_total}
df=DataFrame(bigfan_dict)

#写入excel
print('开始存储')
df.to_csv('Crawled_Dataset\虚拟主播大粉数据.csv',mode='w',encoding='gb18030')
print('存储完毕')   

第 0 个主播共 20 个大粉
第 1 个主播共 20 个大粉
第 2 个主播共 20 个大粉
第 3 个主播共 20 个大粉
第 4 个主播共 20 个大粉
第 5 个主播共 20 个大粉
第 6 个主播共 1 个大粉
第 7 个主播共 13 个大粉
第 8 个主播共 20 个大粉
第 9 个主播共 20 个大粉
第 10 个主播共 20 个大粉
第 11 个主播共 20 个大粉
第 12 个主播共 20 个大粉
第 13 个主播无大粉
第 14 个主播共 20 个大粉
第 15 个主播共 14 个大粉
第 16 个主播共 1 个大粉
第 17 个主播共 20 个大粉
第 18 个主播共 16 个大粉
第 19 个主播共 20 个大粉
第 20 个主播共 20 个大粉
第 21 个主播共 9 个大粉
第 22 个主播共 3 个大粉
第 23 个主播共 20 个大粉
第 24 个主播共 3 个大粉
第 25 个主播共 4 个大粉
第 26 个主播共 20 个大粉
第 27 个主播共 14 个大粉
第 28 个主播共 20 个大粉
第 29 个主播共 20 个大粉
第 30 个主播共 20 个大粉
第 31 个主播共 20 个大粉
第 32 个主播共 20 个大粉
第 33 个主播共 20 个大粉
第 34 个主播共 10 个大粉
第 35 个主播共 20 个大粉
第 36 个主播共 20 个大粉
第 37 个主播共 20 个大粉
第 38 个主播共 17 个大粉
第 39 个主播共 4 个大粉
第 40 个主播共 20 个大粉
第 41 个主播共 7 个大粉
第 42 个主播共 1 个大粉
第 43 个主播共 20 个大粉
第 44 个主播共 20 个大粉
第 45 个主播共 20 个大粉
第 46 个主播共 20 个大粉
第 47 个主播共 2 个大粉
第 48 个主播共 20 个大粉
第 49 个主播共 20 个大粉
第 50 个主播共 20 个大粉
第 51 个主播共 20 个大粉
第 52 个主播共 20 个大粉
第 53 个主播共 20 个大粉
第 54 个主播共 20 个大粉
第 55 个主播共 9 个大粉
第 56 个主播共 3 个大粉
第 57 个主播共 20 个大粉
第 58 个主播共 20 个大粉
第 59 个主播共 2 个大粉
第 60 个主播

第 471 个主播共 20 个大粉
第 472 个主播共 20 个大粉
第 473 个主播无大粉
第 474 个主播共 20 个大粉
第 475 个主播共 1 个大粉
第 476 个主播共 20 个大粉
第 477 个主播共 20 个大粉
第 478 个主播共 20 个大粉
第 479 个主播共 20 个大粉
第 480 个主播共 18 个大粉
第 481 个主播共 20 个大粉
第 482 个主播共 5 个大粉
第 483 个主播共 20 个大粉
第 484 个主播共 20 个大粉
第 485 个主播共 1 个大粉
第 486 个主播共 20 个大粉
第 487 个主播共 20 个大粉
第 488 个主播共 6 个大粉
第 489 个主播共 2 个大粉
第 490 个主播共 20 个大粉
第 491 个主播共 20 个大粉
第 492 个主播共 3 个大粉
第 493 个主播共 1 个大粉
第 494 个主播共 20 个大粉
第 495 个主播共 14 个大粉
第 496 个主播共 10 个大粉
第 497 个主播共 20 个大粉
第 498 个主播共 17 个大粉
第 499 个主播共 20 个大粉
第 500 个主播共 15 个大粉
第 501 个主播共 20 个大粉
第 502 个主播共 12 个大粉
第 503 个主播共 14 个大粉
第 504 个主播共 18 个大粉
第 505 个主播共 4 个大粉
第 506 个主播共 4 个大粉
第 507 个主播共 3 个大粉
第 508 个主播共 12 个大粉
第 509 个主播共 16 个大粉
第 510 个主播共 15 个大粉
第 511 个主播共 20 个大粉
第 512 个主播共 20 个大粉
第 513 个主播共 20 个大粉
第 514 个主播共 20 个大粉
第 515 个主播共 20 个大粉
第 516 个主播共 7 个大粉
第 517 个主播共 1 个大粉
第 518 个主播共 20 个大粉
第 519 个主播共 10 个大粉
第 520 个主播共 20 个大粉
第 521 个主播共 20 个大粉
第 522 个主播共 10 个大粉
第 523 个主播共 20 个大粉
第 524 个主播共 5 个大粉
第 525 个主播共 5 个大粉
第 526 个主播共 7 个大粉
第 527 个主播共 2

第 936 个主播共 20 个大粉
第 937 个主播共 3 个大粉
第 938 个主播共 16 个大粉
第 939 个主播共 20 个大粉
第 940 个主播共 20 个大粉
第 941 个主播共 20 个大粉
第 942 个主播共 20 个大粉
第 943 个主播共 20 个大粉
第 944 个主播共 20 个大粉
第 945 个主播共 20 个大粉
第 946 个主播共 20 个大粉
第 947 个主播共 10 个大粉
第 948 个主播共 7 个大粉
第 949 个主播共 20 个大粉
第 950 个主播共 10 个大粉
第 951 个主播共 20 个大粉
第 952 个主播共 16 个大粉
第 953 个主播共 20 个大粉
第 954 个主播共 11 个大粉
第 955 个主播共 10 个大粉
第 956 个主播共 11 个大粉
第 957 个主播共 20 个大粉
第 958 个主播共 2 个大粉
第 959 个主播共 20 个大粉
第 960 个主播共 20 个大粉
第 961 个主播共 20 个大粉
第 962 个主播共 20 个大粉
第 963 个主播共 20 个大粉
第 964 个主播共 20 个大粉
第 965 个主播共 17 个大粉
第 966 个主播共 20 个大粉
第 967 个主播共 3 个大粉
第 968 个主播共 20 个大粉
第 969 个主播共 20 个大粉
第 970 个主播共 6 个大粉
第 971 个主播共 20 个大粉
第 972 个主播共 13 个大粉
第 973 个主播共 20 个大粉
第 974 个主播共 3 个大粉
第 975 个主播共 8 个大粉
第 976 个主播共 20 个大粉
第 977 个主播共 20 个大粉
第 978 个主播共 20 个大粉
第 979 个主播共 20 个大粉
第 980 个主播共 4 个大粉
第 981 个主播共 20 个大粉
第 982 个主播共 12 个大粉
第 983 个主播共 20 个大粉
第 984 个主播共 5 个大粉
第 985 个主播共 11 个大粉
第 986 个主播共 20 个大粉
第 987 个主播共 20 个大粉
第 988 个主播共 12 个大粉
第 989 个主播共 20 个大粉
第 990 个主播共 20 个大粉
第 991 个主播共 20 个大粉
第

第 1379 个主播共 20 个大粉
第 1380 个主播共 8 个大粉
第 1381 个主播共 20 个大粉
第 1382 个主播共 20 个大粉
第 1383 个主播共 3 个大粉
第 1384 个主播共 20 个大粉
第 1385 个主播共 20 个大粉
第 1386 个主播共 4 个大粉
第 1387 个主播共 9 个大粉
第 1388 个主播共 4 个大粉
第 1389 个主播共 20 个大粉
第 1390 个主播共 8 个大粉
第 1391 个主播共 20 个大粉
第 1392 个主播共 20 个大粉
第 1393 个主播共 20 个大粉
第 1394 个主播共 20 个大粉
第 1395 个主播共 20 个大粉
第 1396 个主播共 20 个大粉
第 1397 个主播共 20 个大粉
第 1398 个主播共 17 个大粉
第 1399 个主播共 20 个大粉
第 1400 个主播共 20 个大粉
第 1401 个主播共 16 个大粉
第 1402 个主播共 20 个大粉
第 1403 个主播共 20 个大粉
第 1404 个主播共 20 个大粉
第 1405 个主播共 4 个大粉
第 1406 个主播共 5 个大粉
第 1407 个主播共 20 个大粉
第 1408 个主播共 16 个大粉
第 1409 个主播共 9 个大粉
第 1410 个主播共 20 个大粉
第 1411 个主播共 20 个大粉
第 1412 个主播共 6 个大粉
第 1413 个主播共 20 个大粉
第 1414 个主播共 20 个大粉
第 1415 个主播共 9 个大粉
第 1416 个主播共 20 个大粉
第 1417 个主播共 4 个大粉
第 1418 个主播共 20 个大粉
第 1419 个主播共 20 个大粉
第 1420 个主播共 15 个大粉
第 1421 个主播共 10 个大粉
第 1422 个主播共 3 个大粉
第 1423 个主播共 20 个大粉
第 1424 个主播共 20 个大粉
第 1425 个主播共 20 个大粉
第 1426 个主播共 15 个大粉
第 1427 个主播共 20 个大粉
第 1428 个主播共 14 个大粉
第 1429 个主播共 20 个大粉
第 1430 个主播共 20 个大粉
第 1431 个主播共 20 个大粉
第 1432

第 1816 个主播共 20 个大粉
第 1817 个主播共 17 个大粉
第 1818 个主播共 20 个大粉
第 1819 个主播共 14 个大粉
第 1820 个主播共 20 个大粉
第 1821 个主播共 20 个大粉
第 1822 个主播共 20 个大粉
第 1823 个主播共 20 个大粉
第 1824 个主播共 20 个大粉
第 1825 个主播共 20 个大粉
第 1826 个主播共 20 个大粉
第 1827 个主播共 9 个大粉
第 1828 个主播共 20 个大粉
第 1829 个主播共 19 个大粉
第 1830 个主播共 20 个大粉
第 1831 个主播共 11 个大粉
第 1832 个主播共 14 个大粉
第 1833 个主播共 19 个大粉
第 1834 个主播共 20 个大粉
第 1835 个主播共 16 个大粉
第 1836 个主播共 13 个大粉
第 1837 个主播共 4 个大粉
第 1838 个主播共 20 个大粉
第 1839 个主播共 20 个大粉
第 1840 个主播共 20 个大粉
第 1841 个主播共 20 个大粉
第 1842 个主播共 15 个大粉
第 1843 个主播共 20 个大粉
第 1844 个主播共 20 个大粉
第 1845 个主播共 6 个大粉
第 1846 个主播共 20 个大粉
第 1847 个主播共 20 个大粉
第 1848 个主播共 14 个大粉
第 1849 个主播共 6 个大粉
第 1850 个主播共 14 个大粉
第 1851 个主播共 20 个大粉
第 1852 个主播共 20 个大粉
第 1853 个主播共 20 个大粉
第 1854 个主播共 20 个大粉
第 1855 个主播共 20 个大粉
第 1856 个主播共 14 个大粉
第 1857 个主播共 20 个大粉
第 1858 个主播共 20 个大粉
第 1859 个主播共 20 个大粉
第 1860 个主播共 20 个大粉
第 1861 个主播共 20 个大粉
第 1862 个主播共 20 个大粉
第 1863 个主播共 9 个大粉
第 1864 个主播共 20 个大粉
第 1865 个主播共 20 个大粉
第 1866 个主播共 19 个大粉
第 1867 个主播共 5 个大粉
第 1868 个主播共 20 个大粉

第 2255 个主播共 12 个大粉
第 2256 个主播共 20 个大粉
第 2257 个主播共 20 个大粉
第 2258 个主播共 20 个大粉
第 2259 个主播共 16 个大粉
第 2260 个主播共 14 个大粉
第 2261 个主播共 20 个大粉
第 2262 个主播共 20 个大粉
第 2263 个主播共 12 个大粉
第 2264 个主播共 14 个大粉
第 2265 个主播共 20 个大粉
第 2266 个主播共 20 个大粉
第 2267 个主播共 20 个大粉
第 2268 个主播共 3 个大粉
第 2269 个主播共 20 个大粉
第 2270 个主播共 20 个大粉
第 2271 个主播共 20 个大粉
第 2272 个主播共 13 个大粉
第 2273 个主播共 12 个大粉
第 2274 个主播共 11 个大粉
第 2275 个主播共 17 个大粉
第 2276 个主播共 20 个大粉
第 2277 个主播共 20 个大粉
第 2278 个主播共 4 个大粉
第 2279 个主播共 2 个大粉
第 2280 个主播共 20 个大粉
第 2281 个主播共 8 个大粉
第 2282 个主播共 6 个大粉
第 2283 个主播共 20 个大粉
第 2284 个主播共 15 个大粉
第 2285 个主播共 5 个大粉
第 2286 个主播共 12 个大粉
第 2287 个主播共 5 个大粉
第 2288 个主播共 20 个大粉
第 2289 个主播共 19 个大粉
第 2290 个主播共 20 个大粉
第 2291 个主播共 18 个大粉
第 2292 个主播共 20 个大粉
第 2293 个主播共 3 个大粉
第 2294 个主播共 1 个大粉
第 2295 个主播共 4 个大粉
第 2296 个主播共 18 个大粉
第 2297 个主播共 7 个大粉
第 2298 个主播共 7 个大粉
第 2299 个主播共 20 个大粉
第 2300 个主播共 16 个大粉
第 2301 个主播共 20 个大粉
第 2302 个主播共 2 个大粉
第 2303 个主播共 3 个大粉
第 2304 个主播共 11 个大粉
第 2305 个主播共 1 个大粉
第 2306 个主播共 8 个大粉
第 2307 个主播共 5 个大粉
第 2308 个主播

第 2697 个主播共 18 个大粉
第 2698 个主播共 20 个大粉
第 2699 个主播共 20 个大粉
第 2700 个主播共 20 个大粉
第 2701 个主播共 7 个大粉
第 2702 个主播共 20 个大粉
第 2703 个主播共 11 个大粉
第 2704 个主播无大粉
第 2705 个主播共 20 个大粉
第 2706 个主播共 12 个大粉
第 2707 个主播共 20 个大粉
第 2708 个主播共 10 个大粉
第 2709 个主播共 20 个大粉
第 2710 个主播共 4 个大粉
第 2711 个主播共 16 个大粉
第 2712 个主播共 10 个大粉
第 2713 个主播共 7 个大粉
第 2714 个主播共 16 个大粉
第 2715 个主播共 5 个大粉
第 2716 个主播共 16 个大粉
第 2717 个主播共 20 个大粉
第 2718 个主播共 5 个大粉
第 2719 个主播共 20 个大粉
第 2720 个主播共 20 个大粉
第 2721 个主播共 5 个大粉
第 2722 个主播共 20 个大粉
第 2723 个主播共 20 个大粉
第 2724 个主播共 1 个大粉
第 2725 个主播共 15 个大粉
第 2726 个主播共 13 个大粉
第 2727 个主播共 19 个大粉
第 2728 个主播共 20 个大粉
第 2729 个主播共 16 个大粉
第 2730 个主播共 19 个大粉
第 2731 个主播共 20 个大粉
第 2732 个主播共 6 个大粉
第 2733 个主播共 6 个大粉
第 2734 个主播共 14 个大粉
第 2735 个主播共 15 个大粉
第 2736 个主播共 20 个大粉
第 2737 个主播共 1 个大粉
第 2738 个主播共 18 个大粉
第 2739 个主播共 4 个大粉
第 2740 个主播共 9 个大粉
第 2741 个主播共 12 个大粉
第 2742 个主播共 6 个大粉
第 2743 个主播共 20 个大粉
第 2744 个主播共 20 个大粉
第 2745 个主播共 20 个大粉
第 2746 个主播共 20 个大粉
第 2747 个主播共 20 个大粉
第 2748 个主播共 20 个大粉
第 2749 个主播共 20 个大粉
第 2750 个主播共

第 3139 个主播共 13 个大粉
第 3140 个主播共 20 个大粉
第 3141 个主播共 4 个大粉
第 3142 个主播共 20 个大粉
第 3143 个主播共 20 个大粉
第 3144 个主播共 8 个大粉
第 3145 个主播共 20 个大粉
第 3146 个主播共 20 个大粉
第 3147 个主播共 6 个大粉
第 3148 个主播共 6 个大粉
第 3149 个主播共 11 个大粉
第 3150 个主播共 20 个大粉
第 3151 个主播共 20 个大粉
第 3152 个主播共 20 个大粉
第 3153 个主播共 3 个大粉
第 3154 个主播共 10 个大粉
第 3155 个主播共 16 个大粉
第 3156 个主播共 20 个大粉
第 3157 个主播共 18 个大粉
第 3158 个主播共 17 个大粉
第 3159 个主播共 10 个大粉
第 3160 个主播共 20 个大粉
第 3161 个主播共 4 个大粉
第 3162 个主播共 20 个大粉
第 3163 个主播共 15 个大粉
第 3164 个主播共 20 个大粉
第 3165 个主播共 11 个大粉
第 3166 个主播共 14 个大粉
第 3167 个主播共 20 个大粉
第 3168 个主播共 20 个大粉
第 3169 个主播共 17 个大粉
第 3170 个主播共 10 个大粉
开始存储
存储完毕
